# Group No 39
## Group Member Names:
	1.	Akilan K. S. L., 2024AB05003
	2.	Devender Kumar, 2024aa05065
	3.	Nagendra Prasad Reddy K. V. S., 2024aa05960
	4.	Sai Venkata Naga Sesh Kumar Ghanta, 2024aa05989
	5.	Piramanayagam P., 2024AB05015


	Text input OR upload .txt file
	Preprocessing: tokenization + lemmatization (NLTK)
	Sentiment model: NLTK VADER (fast, accurate for general English)
	Visualization: color-coded label + bar chart (positive/neutral/negative)

	flask==3.0.3
	nltk==3.9.1

	PART-A
	Task A : Problem Statement:
	
	Develop a simple application that can accurately analyze the sentiment (positive, negative,
	or neutral) of user-provided text. The application should leverage Natural Language
	Processing (NLP) techniques to extract the underlying sentiment from the text.
	Web Interface: (4 Marks)
	1. User Interface: Create an intuitive interface where users can input text or upload
	text files.
	2. Sentiment Display: Visualize the detected sentiment using a clear and
	understandable format (e.g., a bar chart, a color-coded label).
	
	Sentiment Analysis : (4 Marks)
	1. NLP Model Integration: Implement an NLP model (e.g., using libraries like NLTK,
	spaCy, or TensorFlow) capable of performing sentiment analysis.
	2. Text Preprocessing: Apply necessary preprocessing steps (e.g., tokenization, stemming,
	lemmatization) to prepare the text for analysis.
	


README.md
# Sentiment Web App (Flask + NLTK VADER)

A simple sentiment analysis application that:
- Accepts user text or uploaded `.txt` files
- Applies NLP preprocessing (tokenization + lemmatization)
- Predicts sentiment (Positive/Negative/Neutral) using NLTK VADER
- Displays a color-coded label and a bar chart for pos/neu/neg scores

## Setup

### 1) Create venv (recommended)
```bash
python -m venv .venv
# macOS/Linux:
source .venv/bin/activate
# Windows:
# .venv\Scripts\activate

In [1]:
#Install dependencies
import sys
import subprocess

packages = ["flask", "nltk"]

subprocess.check_call([
    sys.executable, "-m", "pip", "install", *packages
])


[notice] A new release of pip is available: 23.2.1 -> 26.0
[notice] To update, run: pip install --upgrade pip


0

In [2]:

"""
Single-file Sentiment Analyzer Web App (Flask + NLTK VADER)

Features:
- Web UI: paste text OR upload a .txt file
- Preprocessing: tokenization + lemmatization (NLTK)
- Sentiment: Positive / Neutral / Negative using VADER
- Visualization: color-coded label + bar chart (Chart.js via CDN)

Run:
  pip install flask nltk
  python sentiment_app.py
Then open:
  http://127.0.0.1:5000
"""

from __future__ import annotations

# Runtime fixes for Jupyter + Flask + NLTK
try:
  sys
except NameError:
  import sys
import os
try:
  import ssl
  ssl._create_default_https_context = ssl._create_unverified_context
except Exception:
  pass

import re
from flask import Flask, request, render_template_string, flash

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer

# -----------------------------
# NLTK downloads (first run)
# -----------------------------
_NLTK_PACKAGES = [
    "punkt",
    "punkt_tab",     # some NLTK builds require this
    "wordnet",
    "omw-1.4",
    "vader_lexicon",
]
for pkg in _NLTK_PACKAGES:
    try:
        # best-effort check; download if missing
        nltk.data.find(pkg)
    except Exception:
        nltk.download(pkg)

lemmatizer = WordNetLemmatizer()
sia = SentimentIntensityAnalyzer()

# -----------------------------
# Flask app
# -----------------------------
app = Flask(__name__)
app.secret_key = "dev-secret-key-change-me"


# -----------------------------
# Preprocessing
# -----------------------------
def preprocess_text(text: str) -> str:
    """
    Preprocess pipeline:
    - strip + normalize whitespace
    - lowercase
    - remove URLs and @mentions
    - keep only letters, spaces, apostrophes
    - tokenize
    - lemmatize
    """
    text = (text or "").strip()
    text = re.sub(r"\s+", " ", text)
    text = text.lower()

    text = re.sub(r"http\S+|www\.\S+", "", text)
    text = re.sub(r"@\w+", "", text)

    # keep letters/spaces/apostrophes
    text = re.sub(r"[^a-z\s']", " ", text)
    text = re.sub(r"\s+", " ", text).strip()

    tokens = word_tokenize(text)
    lemmas = [lemmatizer.lemmatize(tok) for tok in tokens]
    return " ".join(lemmas)


# -----------------------------
# Sentiment prediction
# -----------------------------
def analyze_sentiment(text: str) -> dict:
    """
    Returns dict with:
      label: Positive/Negative/Neutral
      scores: {pos, neu, neg, compound}
      confidence: max(pos, neu, neg) as a simple proxy
    """
    scores = sia.polarity_scores(text)
    c = scores["compound"]

    if c >= 0.05:
        label = "Positive"
    elif c <= -0.05:
        label = "Negative"
    else:
        label = "Neutral"

    confidence = round(max(scores["pos"], scores["neu"], scores["neg"]), 3)

    return {"label": label, "scores": scores, "confidence": confidence}


# -----------------------------
# Helpers
# -----------------------------
def read_uploaded_txt(file_storage) -> str:
    raw = file_storage.read()
    try:
        return raw.decode("utf-8")
    except UnicodeDecodeError:
        return raw.decode("latin-1", errors="ignore")


# -----------------------------
# HTML template (inline)
# -----------------------------
PAGE = """
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8"/>
  <meta name="viewport" content="width=device-width,initial-scale=1"/>
  <title>Sentiment Analyzer</title>
  <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
  <style>
    * { box-sizing: border-box; }
    body {
      font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial, sans-serif;
      margin: 0; background: #f6f7fb; color: #111;
    }
    .container { max-width: 960px; margin: 0 auto; padding: 24px; }
    header { margin-bottom: 16px; }
    h1 { margin: 0 0 6px; }
    p { margin: 0; opacity: 0.8; }

    .card {
      background: white;
      border: 1px solid #e7e7ef;
      border-radius: 14px;
      padding: 16px;
      margin: 14px 0;
      box-shadow: 0 6px 18px rgba(20, 20, 43, 0.06);
    }

    label { display: block; margin: 10px 0 6px; font-weight: 600; }
    textarea, input[type="file"] {
      width: 100%;
      border: 1px solid #d7d7e2;
      border-radius: 10px;
      padding: 10px;
      font-size: 14px;
    }
    button {
      margin-top: 12px;
      border: none;
      border-radius: 12px;
      padding: 10px 14px;
      font-weight: 700;
      cursor: pointer;
      background: #111;
      color: #fff;
    }
    button:hover { opacity: 0.92; }

    .or {
      text-align: center;
      margin: 10px 0;
      font-weight: 700;
      opacity: 0.7;
    }

    .flash { margin: 10px 0; }
    .flash-item {
      background: #fff3cd;
      border: 1px solid #ffeeba;
      padding: 10px;
      border-radius: 10px;
      margin-bottom: 8px;
    }

    .result-row { display: flex; gap: 14px; align-items: center; }
    .meta { font-size: 14px; opacity: 0.9; }

    .badge {
      padding: 10px 14px;
      border-radius: 999px;
      font-weight: 800;
      display: inline-block;
    }
    .badge.positive { background: #e7f7ed; color: #136f2a; border: 1px solid #bfe8ca; }
    .badge.negative { background: #fdecec; color: #9b1c1c; border: 1px solid #f5bcbc; }
    .badge.neutral  { background: #eef2ff; color: #2b3a88; border: 1px solid #cfd7ff; }

    .grid {
      display: grid;
      grid-template-columns: 1fr 1fr;
      gap: 12px;
      margin-top: 10px;
    }
    @media (max-width: 860px) {
      .grid { grid-template-columns: 1fr; }
    }

    .box {
      background: #0b1020;
      color: #f1f5ff;
      border-radius: 12px;
      padding: 12px;
      overflow-x: auto;
      white-space: pre-wrap;
      word-wrap: break-word;
    }
    footer { margin-top: 10px; opacity: 0.7; }
    small code { background: #eee; padding: 2px 6px; border-radius: 8px; }
  </style>
</head>
<body>
  <div class="container">
    <header>
      <h1>Sentiment Analyzer</h1>
      <p>Enter text or upload a <code>.txt</code> file. The app preprocesses and predicts sentiment.</p>
    </header>

    {% with messages = get_flashed_messages() %}
      {% if messages %}
        <div class="flash">
          {% for msg in messages %}
            <div class="flash-item">{{ msg }}</div>
          {% endfor %}
        </div>
      {% endif %}
    {% endwith %}
    <script>
    document.addEventListener("DOMContentLoaded", () => {
      const form = document.querySelector('form[method="POST"]');
      if (!form) return;
      const submitBtn = form.querySelector('button[type="submit"]');
      if (!submitBtn) return;

      const buttonRow = document.createElement("div");
      buttonRow.className = "button-row";
      submitBtn.parentNode.insertBefore(buttonRow, submitBtn);
      buttonRow.appendChild(submitBtn);

      const clearBtn = document.createElement("button");
      clearBtn.type = "button";
      clearBtn.textContent = "Clear";
      clearBtn.className = "secondary-btn";
      buttonRow.appendChild(clearBtn);

      clearBtn.addEventListener("click", () => {
        const textArea = form.querySelector("#text_input");
        if (textArea) textArea.value = "";
        const fileInput = form.querySelector("#text_file");
        if (fileInput) fileInput.value = "";
      });
    });
    </script>
    <style>
    .button-row {
      margin-top: 12px;
      display: flex;
      gap: 10px;
      align-items: center;
    }
    .secondary-btn {
      border: none;
      border-radius: 12px;
      padding: 10px 14px;
      font-weight: 700;
      cursor: pointer;
      background: #f0f0f5;
      color: #111;
    }
    .secondary-btn:hover {
      opacity: 0.92;
    }
    </style>
    <section class="card">
      <form method="POST" enctype="multipart/form-data">
        <label for="text_input">Text input</label>
        <textarea id="text_input" name="text_input" rows="6" placeholder="Type or paste text here...">{{ text_input or "" }}</textarea>

        <div class="or">OR</div>

        <label for="text_file">Upload .txt file</label>
        <input id="text_file" name="text_file" type="file" accept=".txt"/>

        <button type="submit">Analyze Sentiment</button>
      </form>
    </section>

    {% if result %}
      {% set label = result.label %}
      {% if label == "Positive" %}
        {% set badge_class = "badge positive" %}
      {% elif label == "Negative" %}
        {% set badge_class = "badge negative" %}
      {% else %}
        {% set badge_class = "badge neutral" %}
      {% endif %}

      <section class="card">
        <h2>Result</h2>
        <div class="result-row">
          <div class="{{ badge_class }}">{{ label }}</div>
          <div class="meta">
            <div><b>Confidence (proxy):</b> {{ result.confidence }}</div>
            <div><b>Compound:</b> {{ result.scores.compound }}</div>
          </div>
        </div>

        <div class="grid">
          <div>
            <h3>Original text</h3>
            <pre class="box">{{ result.original_text }}</pre>
          </div>
          <div>
            <h3>Preprocessed text</h3>
            <pre class="box">{{ result.cleaned_text }}</pre>
          </div>
        </div>

        <h3>Sentiment score breakdown</h3>
        <canvas id="sentChart" height="120"></canvas>

        <script>
          const data = {
            labels: ["Positive", "Neutral", "Negative"],
            datasets: [{
              label: "Score",
              data: [
                {{ result.scores.pos }},
                {{ result.scores.neu }},
                {{ result.scores.neg }}
              ],
              borderWidth: 1
            }]
          };

          new Chart(
            document.getElementById("sentChart"),
            {
              type: "bar",
              data,
              options: {
                scales: { y: { beginAtZero: true, max: 1 } }
              }
            }
          );
        </script>
      </section>
    {% endif %}

    <footer>
      <small>Model: NLTK VADER • Preprocessing: tokenization + lemmatization</small>
    </footer>
  </div>
</body>
</html>
"""

@app.route("/", methods=["GET", "POST"])
def index():
    result = None
    text_input = ""

    if request.method == "POST":
        text_input = (request.form.get("text_input") or "").strip()
        file = request.files.get("text_file")

        text = ""
        if file and file.filename:
            if not file.filename.lower().endswith(".txt"):
                flash("Please upload a .txt file only.")
                return render_template_string(PAGE, result=None, text_input=text_input)
            text = read_uploaded_txt(file).strip()
        else:
            text = text_input

        if not text:
            flash("Please enter text or upload a .txt file.")
            return render_template_string(PAGE, result=None, text_input=text_input)

        cleaned = preprocess_text(text)
        result = analyze_sentiment(cleaned)
        result["original_text"] = text
        result["cleaned_text"] = cleaned

    return render_template_string(PAGE, result=result, text_input=text_input)


if __name__ == "__main__":
    #app.run(host="127.0.0.1", port=5000, debug=True, use_reloader=False)
    app.run(host="127.0.0.1", port=7614, debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


[nltk_data] Downloading package punkt to /Users/nkunutur/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/nkunutur/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nkunutur/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/nkunutur/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/nkunutur/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
 * Running on http://127.0.0.1:7614
Press CTRL+C to quit
127.0.0.1 - - [04/Feb/2026 19:51:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2026 19:51:00] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/Feb/2026 19:51:12] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2026 19:51:25] "GET / HTTP/1